In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install gensim nltk

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Kwargs/뉴스 크롤링/csv/대한항공_news_data.csv')
print(df.head())

                                      title                 date  \
0               몸 사린 외국인, 그래도 '삼전·하이닉스'는 샀다  2019-02-24 13:08:56   
1  주간추천주]신작 기대감에 엔씨소프트 주목… 현대건설 등도 해외 훈풍 기대  2019-01-13 09:40:58   
2           독립성 확보는 헛말?…사추위에 오너 일가 포함 '수두룩'  2019-02-27 06:01:00   
3                 동부제철우, 상한가 진입.. +29.98% ↑  2019-01-08 09:06:33   
4       외주업체 근로자 또 '안타까운 죽음'…당진 현대제철서 1명 숨져  2019-02-20 21:31:39   

                                             content category  
0  머니투데이 송지유 기자 1월 4조원 이상 순매수→설연휴 후 짙은 관망세깐깐해진 기준...       경제  
1  이데일리 김무연 기자 ‘리니지M’으로 견조한 실적을 올리고 있는 엔씨소프트(0365...       경제  
2  CEO스코어 전수조사의무 설치대상 147개 대기업 중 24곳 (서울연합뉴스) 이승관...       경제  
3  서울경제 오전 9시 4분 현재 동부제철우(016385)가 29.98% 오른 69,8...       경제  
4  문 대통령, 고 김용균 씨 유족 만난 지 이틀 만에 사고 발생(당진연합뉴스) 한종구...       사회  


In [ ]:
# 문서 예시 (실제 데이터에 맞게 수정해야 합니다.)
docs = df['content'].tolist()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# nltk 불용어 사전 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# 한국어 불용어 리스트 (필요에 따라 수정 또는 확장)
stop_words = set(stopwords.words('korean'))  # 'english' 대신 'korean' 불용어 리스트를 사용하거나 직접 정의

# 데이터 전처리 함수
def preprocess_text(text):
    # 소문자 변환 및 특수 문자 제거
    text = re.sub(r'\W', ' ', str(text).lower())
    # 길이가 2 이하인 단어 제거
    text = re.sub(r'\b\w{1,2}\b', '', text)
    # 공백 여러 개를 하나로 줄임
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # 토큰화 및 불용어 제거
    tokens = word_tokenize(text)
    filtered_text = [word for word in tokens if word not in stop_words]
    return filtered_text

preprocessed_docs = [preprocess_text(doc) for doc in docs]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


OSError: No such file or directory: '/root/nltk_data/corpora/stopwords/korean'

In [ ]:
from gensim import corpora, models

# 단어 사전 생성
dictionary = corpora.Dictionary(preprocessed_docs)

# 문서-단어 매트릭스(corpus) 생성
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]

# LDA 모델 훈련
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# 토픽 출력
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)
